Following the success of Chin & Beard Suds Co, we’ve been informed by our distribution company that they’re having some technical difficulties, so we’d like to step in and offer our professional assistance.

# The Scenario

A shipping distribution company transports valuable cargo for customers all around the world using three different types of boat. They keep the basic details for each departure in a table called Departure Details.

<img src="https://3.bp.blogspot.com/-dwNvwjM8RxQ/XJoxZlF4GnI/AAAAAAAAAOI/u_elX3mcfZwF6E29hxElpCLkEgh7qa3TgCLcBGAs/s1600/DepartureDetails.png" alt="Alternative text" />

A snippet of the Departure Details table.
They sell and allocate space on their departures based on the weight and volume of the cargo. They keep the details of each allocation in a table called Allocation Details.

<img src="https://3.bp.blogspot.com/-iLmxdHKcG-s/XJoxZsrzgtI/AAAAAAAAAOU/pGddEQhk3HosX9bia0Zymv9-GmX14cHnQCEwYBhgL/s320/AllocationDetails.png" alt="Alternative text" />

A snippet of the Allocation Details table.
In the last 3 months they’ve begun noticing issues where more weight or volume is being allocated than the boats can actually carry. This means customers are turning up at the port with cargo which the company is then having to turn away and refund. They want to identify and flag departures with weight or volume allocation issues before the day of departure.

# The Requirements


- Figure out how much weight and volume has been allocated for each departure in the Allocation Details table.
-  Find a way to flag departures on the Departure Details table which have had too much cargo weight or volume allocated.

# The Output

- One table designed to replace the original Departure Details table.
- 8 fields total (4 original & 4 new). 
- 58 rows.
- Two new fields called [Allocated Weight] & [Allocated Volume] which contain the allocated weight and volume for each departure.
- Two more new fields called [Overweight?] & [Overvolume?] which contain TRUE or FALSE depending on if there’s an issue.

In [193]:
import pandas as pd
import numpy as np

In [194]:
df1 = pd.read_excel('Week7Challenge.xlsx', sheet_name='Departure Details')
df1.head()

,Ship ID,Departure Date,Max Weight,Max Volume
0,Tug-05,2019-03-27,500,50
1,Freighter-04,2019-03-31,5000,500
2,Freighter-03,2019-04-01,5000,500
3,Freighter-01,2019-04-06,5000,500
4,Tanker-02,2019-04-07,1000,200


In [195]:
df2 = pd.read_excel('Week7Challenge.xlsx', sheet_name='Allocation Details')
df2.head()

,Salesperson,Departure ID,Date Logged,Product Type,Weight Allocated,Volume Allocated
0,Daisy Steiner,Tanker-05-04-05-2019,2019-02-21,Spurs Jerseys,111,10
1,Tim Bisley,Freighter-01-18-06-2019,2019-01-14,Phone Catalogues,756,48
2,Daisy Steiner,Tanker-04-27-04-2019,2019-01-26,Oil,220,28
3,Colin Chien,Freighter-02-10-04-2019,2019-03-11,Rubber Ducks,536,20
4,Colin Chien,Tanker-05-20-05-2019,2019-01-27,Vinyl Records,148,30


In [196]:
agg_df2 = df2.groupby(['Departure ID']).agg({'Weight Allocated':'sum','Volume Allocated':'sum'})
agg_df2.reset_index(inplace=True)
agg_df2.head()

,Departure ID,Weight Allocated,Volume Allocated
0,Freighter-01-06-04-2019,1336,112
1,Freighter-01-18-06-2019,4580,556
2,Freighter-02-03-06-2019,2976,112
3,Freighter-02-10-04-2019,3972,396
4,Freighter-02-11-06-2019,1056,102


In [197]:
df1['Year'] = df1['Departure Date'].dt.year.astype(str)
df1['Month'] = df1['Departure Date'].dt.month
df1['Day'] = df1['Departure Date'].dt.day
df1.head()

,Ship ID,Departure Date,Max Weight,Max Volume,Year,Month,Day
0,Tug-05,2019-03-27,500,50,2019,3,27
1,Freighter-04,2019-03-31,5000,500,2019,3,31
2,Freighter-03,2019-04-01,5000,500,2019,4,1
3,Freighter-01,2019-04-06,5000,500,2019,4,6
4,Tanker-02,2019-04-07,1000,200,2019,4,7


In [198]:
for index, series in df1.iterrows():
    if df1.loc[index,'Day'] >= 10:
        df1.loc[index,'Day'] = str(df1.loc[index,'Day'])
    else:
        df1.loc[index,'Day'] = '0' + str(df1.loc[index,'Day'])

    if df1.loc[index,'Month'] >= 10:
        df1.loc[index,'Month'] = str(df1.loc[index,'Month'])
    else:
        df1.loc[index,'Month'] = '0' + str(df1.loc[index,'Month'])
df1.head()

,Ship ID,Departure Date,Max Weight,Max Volume,Year,Month,Day
0,Tug-05,2019-03-27,500,50,2019,03,27
1,Freighter-04,2019-03-31,5000,500,2019,03,31
2,Freighter-03,2019-04-01,5000,500,2019,04,01
3,Freighter-01,2019-04-06,5000,500,2019,04,06
4,Tanker-02,2019-04-07,1000,200,2019,04,07


In [199]:
df1['Departure ID'] = df1['Ship ID'] + '-' + df1['Day'] + '-' + df1['Month'] + '-' + df1['Year']
df1.drop(columns=['Year','Month','Day'], inplace=True)
df1.head()

,Ship ID,Departure Date,Max Weight,Max Volume,Departure ID
0,Tug-05,2019-03-27,500,50,Tug-05-27-03-2019
1,Freighter-04,2019-03-31,5000,500,Freighter-04-31-03-2019
2,Freighter-03,2019-04-01,5000,500,Freighter-03-01-04-2019
3,Freighter-01,2019-04-06,5000,500,Freighter-01-06-04-2019
4,Tanker-02,2019-04-07,1000,200,Tanker-02-07-04-2019


In [200]:
df = pd.merge(df1, agg_df2, how ='left', on='Departure ID')
df.head()

,Ship ID,Departure Date,Max Weight,Max Volume,Departure ID,Weight Allocated,Volume Allocated
0,Tug-05,2019-03-27,500,50,Tug-05-27-03-2019,707,68
1,Freighter-04,2019-03-31,5000,500,Freighter-04-31-03-2019,616,106
2,Freighter-03,2019-04-01,5000,500,Freighter-03-01-04-2019,2392,232
3,Freighter-01,2019-04-06,5000,500,Freighter-01-06-04-2019,1336,112
4,Tanker-02,2019-04-07,1000,200,Tanker-02-07-04-2019,809,110


In [201]:
df.drop(columns=['Departure ID'])
df['Overweight?'] = np.where(df['Weight Allocated'] > df['Max Weight'], 1, 0)
df['Overvolume?'] = np.where(df['Volume Allocated'] > df['Max Volume'], 1, 0)
df.head()

,Ship ID,Departure Date,Max Weight,Max Volume,Departure ID,Weight Allocated,Volume Allocated,Overweight?,Overvolume?
0,Tug-05,2019-03-27,500,50,Tug-05-27-03-2019,707,68,1,1
1,Freighter-04,2019-03-31,5000,500,Freighter-04-31-03-2019,616,106,0,0
2,Freighter-03,2019-04-01,5000,500,Freighter-03-01-04-2019,2392,232,0,0
3,Freighter-01,2019-04-06,5000,500,Freighter-01-06-04-2019,1336,112,0,0
4,Tanker-02,2019-04-07,1000,200,Tanker-02-07-04-2019,809,110,0,0


In [202]:
result = df.drop(columns=['Departure ID'])
result.head(100)


,Ship ID,Departure Date,Max Weight,Max Volume,Weight Allocated,Volume Allocated,Overweight?,Overvolume?
0,Tug-05,2019-03-27,500,50,707,68,1,1
1,Freighter-04,2019-03-31,5000,500,616,106,0,0
2,Freighter-03,2019-04-01,5000,500,2392,232,0,0
3,Freighter-01,2019-04-06,5000,500,1336,112,0,0
4,Tanker-02,2019-04-07,1000,200,809,110,0,0
5,Tanker-04,2019-04-10,1000,200,257,58,0,0
6,Freighter-02,2019-04-10,5000,500,3972,396,0,0
7,Tug-05,2019-04-10,500,50,222,27,0,0
8,Freighter-03,2019-04-10,5000,500,3476,456,0,0
9,Freighter-04,2019-04-12,5000,500,2120,176,0,0
